In [1]:
from rag import Custom_Rag


## EVALUATION METRICS - LLM JUDGE

![image](rag_eval_overview-c0c3c2ac03f44b2b67c58ab67f32d82a.png)

In [2]:
from langchain import hub
from langchain_openai import ChatOpenAI

# Grade prompt
grade_prompt_answer_accuracy = prompt = hub.pull("langchain-ai/rag-answer-vs-reference")

def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """

    # Get question, ground truth answer, RAG chain answer
    input_question = example.inputs.get("prompt")
    reference = example.outputs.get("answer")
    prediction = run.outputs.get("response")

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_answer_accuracy | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_v_reference_score", "score": score}

# Grade prompt
grade_prompt_answer_helpfulness = prompt = hub.pull("langchain-ai/rag-answer-helpfulness")

def answer_helpfulness_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer helpfulness
    """

    # Get question, ground truth answer, RAG chain answer
    input_question = example.inputs.get("prompt")
    prediction = run.outputs.get("response")

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_answer_helpfulness | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_helpfulness_score", "score": score}

# Prompt
grade_prompt_hallucinations = prompt = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """

    # RAG inputs
    input_question = example.inputs.get("prompt")
    contexts = run.outputs.get("context")

    # RAG answer
    prediction = run.outputs.get("response")
    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

# Grade prompt
grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """

    # RAG inputs
    input_question = example.inputs.get("prompt")
    contexts = run.outputs.get("context")

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [3]:
list_of_models=["gpt-3.5-turbo", "gpt-4o","gpt-4-turbo","gpt-3.5-turbo-0125","gpt-4"]
dataset_name="full-dataset"

In [4]:
from langsmith.evaluation import  evaluate
for model in list_of_models:
    xperiment_results = evaluate(
    lambda inputs: Custom_Rag(model=model).main(inputs["prompt"]),
    data=dataset_name,
    evaluators=[answer_evaluator, answer_helpfulness_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix=f"{model} rag eval",
    # Any experiment metadata can be specified here
    metadata={
        "variant": "LCEL context, gpt-4-turbo",
    },
    num_repetitions=3
)

c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-3.5-turbo rag eval-57d25218' at:
https://smith.langchain.com/o/b47753b3-6d84-5fa3-b240-64adbbe1c7be/datasets/88a61045-2ca2-47c7-a3b1-91149cab59c1/compare?selectedSessions=4a01ce54-efba-497e-bbc7-a69e08d7fb25




0it [00:00, ?it/s]

client=<openai.resources.chat.completions.Completions object at 0x0000026E780C86E0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026E78123290> temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''
client=<openai.resources.chat.completions.Completions object at 0x0000026E7809DA90> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026E79438EC0> temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''
client=<openai.resources.chat.completions.Completions object at 0x0000026E780EED50> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026E79458C50> temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''
client=<openai.resources.chat.completions.Completions object at 0x0000026E7809F620> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026E79498B90> temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''


c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


client=<openai.resources.chat.completions.Completions object at 0x0000026E81B8B320> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026E81B886B0> model_name='gpt-4' temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''
client=<openai.resources.chat.completions.Completions object at 0x0000026E919CBDA0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026E93B6B5C0> model_name='gpt-4' temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''


c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


client=<openai.resources.chat.completions.Completions object at 0x0000026E8569D1F0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026E78116A50> model_name='gpt-4' temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''
client=<openai.resources.chat.completions.Completions object at 0x0000026E93B6A000> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026E935062D0> model_name='gpt-4' temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''


c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c

## langsmith evaluation report


https://smith.langchain.com/public/5c59d4a8-4159-481d-9dc3-e55be829246c/d